#**Taller Regresión Logistica**
###Integrantes:
- Daniel Esteban Méndez Díaz
- Juan David Godoy Valencia
- Johan Santiago Ramos Duarte
----

# Sobrevivientes del Titanic

### Autor: Edwin Andrés Niño Velásquez

El 15 de abril de 1912, el Titanic naufragó después de chocar con un iceberg. Debido a la insuficiencia de botes salvavidas, 1502 de los 2224 pasajeros y tripulantes murieron.

Aunque la suerte jugó un papel en la supervivencia de los viajeros, algunos grupos pudieron tener mayores posibilidades de sobrevivir que otros.

Se desea construir un modelo de ML que permita dar respuesta a la pregunta: ¿Qué tipo de persona tuvo más posibilidades de sobrevivir?. Para esto, se cuenta con información como nombre, edad, sexo, clase del tiquete, ciudad de embarque, entre otros.

## Importar librerías
Revisar las librerías/funciones importadas e intentar inferir el propósito de cada una.
----
- **numpy (import numpy as np):** para realizar cálculos lógicos y matemáticos sobre cuadros y matrices (NumPy -, n.d.).
- **pandas (import pandas as pd):**Muy usada para el analisis de datos, aqui para leer el archivo de Dropbox y crear el dataframe.
- **LogisticRegression (from sklearn.linear_model import LogisticRegression):**  Implementa la regresión logística, un algoritmo de clasificación visto en clase.
- **train_test_split (from sklearn.model_selection import train_test_split):** Divide un conjunto de datos en subconjuntos de entrenamiento, validación y prueba, facilitando la evaluación del modelo Machine Learning, en este caso de la Regresión Logísctica.
- **lassification_report, confusion_matrix (from sklearn.metrics import classification_report, confusion_matrix):** Nos permite visualizar la matriz de confusión y el reporte para evaluar el modelo.
- **math (import math):** Proporciona acceso a las funciones matemáticas básicas.
- **requests (import requests):**  Permite enviar solicitudes HTTP de manera sencilla y manejar respuestas, útil para interactuar con APIs web y descargar contenido de internet J2logo (2022). En este caso para acceder al archivo que se encuentra en Drobox.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import math
import requests

## Cargar el conjunto de datos
Dar un vistazo a los datos. ¿De qué características se dispone? ¿Qué particularidades encuentra en los datos? ¿Cuántos ejemplos se tiene?

SibSp: Número de hermanos/esposo(a) abordo.

Parch: Número de padres/hijos abordo

Embarked: Ciudad de embarque.

También, puede descargar el archivo del enlace y explorarlo usando excel.

In [ ]:
url = "https://www.dropbox.com/s/g19rqwd53co5dh1/titanic.csv?dl=1"
response = requests.get(url)
filename = 'titanic.csv'
with open(filename, 'wb') as file:
    file.write(response.content)
data = pd.read_csv(filename, sep=';')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,712.833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30,C148,C


## Descripción del conjunto de datos
Entender las variables e identificar valores anómalos.
----
- Hay valores anomalos como en "Age" ya que hay datos faltantes (714) en comparación de los 891 que hay en total,

In [ ]:
print(data.columns)
data.describe()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,PassengerId,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000


## Filtrar y transformar características y etiquetas

#### Identificar y eliminar columnas irrelevantes
¿Cuáles columnas son irrelevantes? Elimínelas.
- **PassengerId:** Es un identificador único para cada pasajero y no aporta información útil para la predicción.
- **Ticket:** El número de ticket en sí no proporciona información relevante para la predicción.
- **Name:** El nombre completo no es útil para este contexto en específico.
- **Cabin:** Contiene muchos valores faltantes y, aunque puede tener algún valor predictivo, se considera irrelevante para el análisis actual.

In [ ]:
print(data['PassengerId'].nunique()) #Irrelevante
print(data['Ticket'].nunique()) #Irrelevante
print(data['Name'].nunique()) #Irrelevante
print(data['Pclass'].nunique())
print(data['SibSp'].nunique())
print(data['Parch'].nunique())
print(data['Fare'].nunique())
print(data['Cabin'].nunique()) #Irrelevante
#Número de datos faltantes en el campo Cabin
print(len(data[data['Cabin'].isna()]))

#Eliminar columna <nombre_columna>
del data['PassengerId']
del data['Ticket']
del data['Name']
del data['Cabin']

data

891
681
891
3
7
7
247
147
687


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.25,S
1,1,1,female,38.0,1,0,712.833,C
2,1,3,female,26.0,0,0,7.925,S
3,1,1,female,35.0,1,0,53.1,S
4,0,3,male,35.0,0,0,8.05,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13,S
887,1,1,female,19.0,0,0,30,S
888,0,3,female,NaN,1,2,23.45,S
889,1,1,male,26.0,0,0,30,C


#### **Procesar variables restantes**
Analice cada una de las transformaciones realizadas a las variables e intente explicar el propósito de cada una. Analice el conjunto de datos resultante.
* **Convertir sexo a numérico:** Convertir la variable categórica Sex a valores numéricos (0 para hombres, 1 para mujeres) para que la Regresión Logística  pueda procesarla.

  Para entender el código de lo que se hace con las demás variables tenemos que tener claro el concepto de normalizar y escalar. "Es conveniente normalizar las variables numéricas cuando vamos a utilizar modelos de Machine Learning que asumen que los datos de entrada siguen una distribución normal"(Mejores Transformaciones De Variables Numéricas Para ML, 2024).

- **Normalizar Pclass:** Escalar la variable Pclass (clase de pasajero) para que los valores estén en el rango de 0 a 1. Esto ayuda a estandarizar las variables y facilita el aprendizaje del modelo.
- **Normalizar SibSp:** Escalar la variable SibSp (número de hermanos/esposos a bordo) para que esté en el rango de 0 a 1, similar a Pclass.
- **Normalizar Parch:** Escalar la variable Parch (número de padres/hijos a bordo) para que esté en el rango de 0 a 1, asegurando consistencia en la escala de las variables.
- **Convertir Fare a números:** Eliminar el punto decimal y convertir la variable Fare a tipo float para su procesamiento posterior.
- **Normalizar Fare:** Escalar la tarifa (Fare) para que sus valores estén en el rango de 0 a 1.
- **One-hot encoding de punto de embarcación:**  Convertir la variable categórica Embarked en variables binarias (one-hot encoding) para cada puerto de embarque (C, Q, S). Esto permite que los algoritmos de Machine Learning manejen estas categorías de manera efectiva (GeeksforGeeks, 2024).
- **Convertir edad a numérico:** Asegurarse de que la variable Age sea del tipo float para su posterior procesamiento.
- **Imputar datos faltantes de edad con la media:**  Rellenar los valores faltantes en la variable Age con la media de las edades. Esto evita problemas que podrían surgir debido a valores nulos.
- **Normalizar edad:** Escalar la variable Age para que sus valores estén en el rango de 0 a 1.
###**Análisis del Conjunto de Datos Resultante**
Después de las transformaciones, el conjunto de datos contiene solo variables numéricas y está normalizado. Esto facilita la aplicación del algoritmo de la regresión logística.

In [ ]:
#Convertir sexo a numérico
data['Sex'].replace({'male':0, 'female':1}, inplace=True)
#Normalizar Pclass
data['Pclass'] = data['Pclass'] / data['Pclass'].max()
#Normalizar SibSp (Número de hermanos/esposo(a) abordo)
data['SibSp'] = data['SibSp'] / data['SibSp'].max()
#Normalizar Parch (Número de padres/hijos abordo)
data['Parch'] = data['Parch'] / data['Parch'].max()
#Convertir Fare (Costo del tiquete) a números
data['Fare'] = data['Fare'].apply(lambda x:str(x).replace('.', '')).astype(float)
#Normalizar Fare (Costo del tiquete)
data['Fare'] = data['Fare'] / data['Fare'].max()
#One-hot encoding de punto de embarcación (C = Cherbourg, Q = Queenstown, S = Southampton)
data['E_C'] = (data['Embarked']=='C').replace({True:1, False:0})
data['E_Q'] = (data['Embarked']=='Q').replace({True:1, False:0})
data['E_S'] = (data['Embarked']=='S').replace({True:1, False:0})
del data['Embarked']
#Convertir edad a numérico
data['Age'] = data['Age'].astype(float)
#Imputar datos faltantes de edad con la media
data['Age'] = data['Age'].fillna(data['Age'].mean())
#Normalizar edad
data['Age'] = data['Age'] / data['Age'].max()
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,E_C,E_Q,E_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,0.769547,0.352413,0.371239,0.065376,0.063599,0.024918,0.188552,0.086420,0.722783
std,0.486592,0.278690,0.477990,0.162525,0.137843,0.134343,0.080246,0.391372,0.281141,0.447876
min,0.000000,0.333333,0.000000,0.005250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.666667,0.000000,0.275000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.371239,0.000000,0.000000,0.000512,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.437500,0.125000,0.000000,0.015412,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



## Analizar correlación entre variables
Una característica con una correlación moderada o fuerte con las etiquetas con seguridad será relevante en el modelo de clasificación.

Dos características con una correlación fuerte entre sí proveen información redundante.

- **¿Qué información destacada se puede observar en la matriz de correlación?**

La correlación se entiende, cuando es 1 es una correlación positiva perfecta y cuando es -1 es una correlación negativa perfecta, para cuando da 0, la correlación no existe (Ortega, 2023).

*   **¿Qué características tienen una correlación moderada o fuerte con la etiqueta?**

Sex tiene una correlación positiva moderada con Survived (0.543). Esto sugiere que el género femenino tiene una mayor probabilidad de supervivencia.

Pclass tiene una correlación negativa moderada con Survived (-0.338). Esto indica que los pasajeros en clases más altas (clase 1) tienen mayores tasas de supervivencia.

Fare tiene una correlación positiva débil con Survived (0.185). Esto sugiere que los pasajeros que pagaron tarifas más altas tienen una mayor probabilidad de supervivencia.


E_C (embarcados en Cherbourg) tiene una correlación positiva débil con Survived (0.168). Esto sugiere que los pasajeros que embarcaron en Cherbourg tienen una ligera ventaja en la supervivencia.

E_S (embarcados en Southampton) tiene una correlación negativa débil con Survived (-0.156). Esto sugiere que los pasajeros que embarcaron en Southampton tienen una menor probabilidad de supervivencia.

* **¿Los datos sugieren alguna relación (aunque sea débil) entre la edad y el poder adquisitivo de una persona?**

La correlación entre Age y Fare es 0.083297, lo cual es bastante débil. Esto indica que no hay una relación significativa entre la edad y el costo del boleto, y por lo tanto, no se puede inferir una relación clara entre la edad y el poder adquisitivo de una persona.

*   **¿En qué ciudad parece vivir la gente más adinerada?**

La correlación entre Fare y E_C es 0.294865, indicando que los pasajeros que embarcaron en Cherbourg tienden a pagar tarifas más altas. Por lo tanto, Cherbourg parece ser la ciudad de origen de los pasajeros más adinerados.

*   **¿Qué otras variables están correlacionadas de manera moderada o fuerte?**

SibSp y Parch: Tienen una correlación positiva moderada (0.414838). Esto indica que los pasajeros que viajaban con hermanos/esposos a bordo también tienden a viajar con padres/hijos.

Pclass y Age: Tienen una correlación negativa moderada (-0.331339). Esto sugiere que los pasajeros de clases más bajas tienden a ser más jóvenes.

E_S y E_C: Tienen una fuerte correlación negativa (-0.778359). Esto se debe a que estas variables son parte de un conjunto de codificación one-hot y no pueden ser altas al mismo tiempo.

In [ ]:
data.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,E_C,E_Q,E_S
Survived,1.000000,-0.338481,0.543351,-0.069809,-0.035322,0.081629,0.185894,0.168240,0.003650,-0.155660
Pclass,-0.338481,1.000000,-0.131900,-0.331339,0.083081,0.018443,-0.296741,-0.243292,0.221009,0.081720
Sex,0.543351,-0.131900,1.000000,-0.084153,0.114631,0.245489,0.092542,0.082853,0.074115,-0.125722
Age,-0.069809,-0.331339,-0.084153,1.000000,-0.232625,-0.179191,0.083297,0.032024,-0.013855,-0.027121
SibSp,-0.035322,0.083081,0.114631,-0.232625,1.000000,0.414838,-0.027361,-0.059528,-0.026354,0.070941
Parch,0.081629,0.018443,0.245489,-0.179191,0.414838,1.000000,0.046245,-0.011069,-0.081228,0.063036
Fare,0.185894,-0.296741,0.092542,0.083297,-0.027361,0.046245,1.000000,0.294865,-0.078494,-0.206836
E_C,0.168240,-0.243292,0.082853,0.032024,-0.059528,-0.011069,0.294865,1.000000,-0.148258,-0.778359
E_Q,0.003650,0.221009,0.074115,-0.013855,-0.026354,-0.081228,-0.078494,-0.148258,1.000000,-0.496624
E_S,-0.155660,0.081720,-0.125722,-0.027121,0.070941,0.063036,-0.206836,-0.778359,-0.496624,1.000000


## Separar características de etiquetas
Divide en X y y las características y etiquetas. Calcule el número de ejemplos en cada clase.

####**Análisis**
Distribución de Supervivencia:
No sobrevivieron (Clase 0): 549 ejemplos, representando aproximadamente el 61.62% del total.
Sobrevivieron (Clase 1): 342 ejemplos, representando aproximadamente el 38.38% del total.

In [ ]:
y = data['Survived']
del data['Survived']
X = data
print(y.shape)
print(X.shape)
print(y.describe())
# Calcular el número de ejemplos en cada clase
print('Clase 0:', y[y==0].shape, len(y[y==0])/len(y)) #Vivo
print('Clase 1:', y[y==1].shape, len(y[y==1])/len(y)) #Muerto

(891,)
(891, 9)
count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Clase 0: (549,) 0.6161616161616161
Clase 1: (342,) 0.3838383838383838


## Crear conjunto de entrenamiento y conjunto de prueba
70% entrenamiento, 30% prueba

¿Cuántos ejemplos hay en cada conjunto?

* De entrenamiento vamos a contar con 623 datos y para validación tendremos el 30% que son 268 datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=0)
print('Entrenamiento: ', y_train.shape)
print('Prueba: ', y_test.shape)

Entrenamiento:  (623,)
Prueba:  (268,)


## Línea base
Para evaluar el desempeño del modelo de clasificación, se define un clasificador que etiqueta todos los ejemplos del conjunto de prueba como de la clase mayoritaria.

In [ ]:
y_baseline = pd.Series([0]*len(y_test))
print(y_baseline)

0      0
1      0
2      0
3      0
4      0
      ..
263    0
264    0
265    0
266    0
267    0
Length: 268, dtype: int64


## Evaluar el desempeño de la línea base
Analizar matriz de confusión, exactitud, precisión y tasa de recuperación.

###**Análisis:**
**Matriz de Confusión:**

- Verdaderos Negativos (Clase 0 correctamente predicha): 163
- Falsos Negativos (Clase 1 incorrectamente predicha como Clase 0): 105

**Informe de Clasificación:**

- Precisión para Clase 0: 0.63 (163/268)
- Recall para Clase 0: 1.00 (163/163)
- F1-Score para Clase 0: 0.77
- Precisión para Clase 1: 0.00 (no hay verdaderos positivos para Clase 1)
- Recall para Clase 1: 0.00
- F1-Score para Clase 1: 0.00
- Precisión Global (Accuracy): 0.63

In [ ]:
print(confusion_matrix(y_test, y_baseline))
print(classification_report(y_test, y_baseline))

[[168   0]
 [100   0]]
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       168
           1       0.00      0.00      0.00       100

    accuracy                           0.63       268
   macro avg       0.31      0.50      0.39       268
weighted avg       0.39      0.63      0.48       268



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Entrenar un modelo de regresión logística
Analice los coeficientes obtenidos (signo y magnitud). Dé una interpretación en el contexto del problema.

La regresión logística se utiliza para modelar la probabilidad de una variable de respuesta binaria (Pérez et al., 2010), en este caso, si un pasajero sobrevivió o no, en función de una o más variables independientes (características). Después de entrenar el modelo, los coeficientes obtenidos nos indican la influencia de cada característica en la probabilidad de supervivencia.

### **Interpretación de los Coeficientes**
Los coeficientes indican la dirección y magnitud de la relación entre cada característica y la probabilidad de supervivencia. Un coeficiente positivo indica que, a medida que la característica aumenta, también lo hace la probabilidad de supervivencia. Un coeficiente negativo indica lo contrario (SPSS Statistics Subscription - Classic, n.d.).
- **Pclass (w = -2.543296):** Los pasajeros de clase más baja, tienen una menor probabilidad de sobrevivir. La magnitud del coeficiente indica una fuerte relación negativa.
- **Sex (w = 2.489201):** Ser mujer aumenta la probabilidad de supervivencia en comparación con ser hombre. Este coeficiente tiene una magnitud grande, indicando una fuerte relación positiva.
- **Age (w = -1.852322):** A medida que la edad aumenta, la probabilidad de supervivencia disminuye. Este coeficiente negativo sugiere que los pasajeros más jóvenes tenían más probabilidades de sobrevivir.
- **SibSp (w = -1.470781):**
Un mayor número de hermanos o esposos a bordo está asociado con una menor probabilidad de supervivencia.
- **Parch (w = -0.453228):**
Un mayor número de padres o hijos a bordo también está asociado con una menor probabilidad de supervivencia, aunque la relación no es tan fuerte como para SibSp.
- **Fare (w = 0.840554):**
Los pasajeros que pagaron tarifas más altas tienen una mayor probabilidad de sobrevivir. Este coeficiente positivo sugiere una relación directa entre el poder adquisitivo y la probabilidad de supervivencia.
- **E_C (w = 0.111076):**
Embarcar en Cherburgo (C) tiene una ligera asociación positiva con la probabilidad de supervivencia.
- **E_Q (w = 0.018381):**
Embarcar en Queenstown (Q) muestra una relación muy débil y casi neutral con la probabilidad de supervivencia.
- **E_S (w = -0.380582):**
Embarcar en Southampton (S) está asociado con una menor probabilidad de supervivencia en comparación con las otras ciudades de embarque.

 Los resultados muestran que ser mujer, estar en una clase superior, y pagar una tarifa más alta aumentan significativamente las probabilidades de supervivencia.

In [ ]:
model = LogisticRegression(n_jobs=-1)
model.fit(X_train, y_train)
w = np.hstack([np.array([model.intercept_[0]]), model.coef_[0]])
print('w_0',model.intercept_[0])
coefs = pd.DataFrame(w[1:], columns=['w'])
coefs['feat_name'] = X.columns
coefs

w_0 1.5565150328235229


,w,feat_name
0,-2.552078,Pclass
1,2.488815,Sex
2,-1.855318,Age
3,-1.464406,SibSp
4,-0.450415,Parch
5,0.832464,Fare
6,0.115509,E_C
7,0.023718,E_Q
8,-0.376476,E_S


## Clasificar los ejemplos de prueba
Clasifique los objetos del conjunto de prueba usando el modelo entrenado y evalúe el desempeño del clasificador. Use un umbral de 0.5.

Analice la matriz de confusión y las medidas de exactitud, precisión y tasa de recuperación.
- **Probabilidades Predichas:**
La primera columna muestra la probabilidad de que el pasajero no sobreviva (clase 0) y la segunda columna muestra la probabilidad de que sobreviva (clase 1).
- **Predicciones Basadas en el Umbral:**
Convertimos las probabilidades en predicciones de clase usando el umbral de 0.5. Si la probabilidad de supervivencia (p1) es mayor o igual a 0.5, predice 1 (sobrevive); de lo contrario, predice 0 (no sobrevive).
- **Resumen de Valores Reales y Predicciones:**
Estos resúmenes proporcionan una visión general de la distribución de las clases en los valores reales y las predicciones.

###**Matriz de Confusión:**

- Verdaderos Negativos (Clase 0 correctamente predicha): 163
- Falsos Negativos (Clase 1 incorrectamente predicha como Clase 0): 105

###**Informe de Clasificación:**
###Precisión (Precision):

- Clase 0 (No sobrevive): 0.84
- Clase 1 (Sobrevive): 0.72
La precisión es mayor para la clase 0 que para la clase 1, indicando que el modelo es mejor prediciendo a los pasajeros que no sobreviven.
###Tasa de recuperación
- Clase 0 (No sobrevive): 0.83
- Clase 1 (Sobrevive): 0.73

  De todos los pasajeros que realmente no sobrevivieron, el modelo identificó correctamente el 83%. De todos los pasajeros que realmente sobrevivieron, el modelo identificó correctamente el 73%. La tasa de recuperación es ligeramente mejor para la clase 0.
###Puntaje F1 (F1-score):

- Clase 0 (No sobrevive): 0.84
- Clase 1 (Sobrevive): 0.73

  El puntaje F1 es la media armónica de la precisión y la tasa de recuperación. Es un buen indicador del balance entre las dos métricas. Nuevamente, el modelo se desempeña mejor en predecir la clase 0.

###Exactitud (Accuracy):
- Exactitud general: 0.79 (79%)

  El modelo predijo correctamente el 79% de los ejemplos en el conjunto de prueba. Esto sugiere un desempeño razonablemente bueno, aunque hay margen de mejora.
  

In [ ]:
threshold = 0.5
proba = model.predict_proba(X_test)
print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
print(real_vs_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[0.82423789 0.17576211]
 [0.88702264 0.11297736]
 [0.87399195 0.12600805]
 [0.06438006 0.93561994]
 [0.30771104 0.69228896]
 [0.5888881  0.4111119 ]
 [0.10528282 0.89471718]
 [0.09823666 0.90176334]
 [0.44856142 0.55143858]
 [0.28257159 0.71742841]
 [0.89522937 0.10477063]
 [0.26118634 0.73881366]
 [0.84030322 0.15969678]
 [0.16037825 0.83962175]
 [0.0527164  0.9472836 ]
 [0.30898749 0.69101251]
 [0.87033731 0.12966269]
 [0.77197686 0.22802314]
 [0.89876802 0.10123198]
 [0.72132131 0.27867869]
 [0.71885205 0.28114795]
 [0.09026436 0.90973564]
 [0.83862591 0.16137409]
 [0.58889888 0.41110112]
 [0.34920219 0.65079781]
 [0.10673003 0.89326997]
 [0.89103307 0.10896693]
 [0.35281479 0.64718521]
 [0.21204931 0.78795069]
 [0.40826372 0.59173628]
 [0.8363473  0.1636527 ]
 [0.30833565 0.69166435]
 [0.88302418 0.11697582]
 [0.58888613 0.41111387]
 [0.90169438 0.09830562]
 [0.48019693 0.51980307]
 [0.9144458  0.0855542 ]
 [0.76326988 0.23673012]
 [0.75047205 0.24952795]
 [0.85491523 0.14508477]


## Probar distintos umbrales
¿Qué umbral en {0.2, 0.4, 0.5, 0.6, 0.8} da una mayor exactitud?

#**Umbral de 0.2**

- Exactitud (Accuracy): 0.69 (69%)
###Clase 0 (No sobrevive):
- Precisión: 0.88
- Tasa de recuperación (Recall): 0.58
- F1-score: 0.70
###Clase 1 (Sobrevive):
- Precisión: 0.55
- Tasa de recuperación (Recall): 0.87
- F1-score: 0.65

In [ ]:
threshold = 0.2
proba = model.predict_proba(X_test)
#print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
#print(real_vs_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Real y
 count    268.000000
mean       0.373134
std        0.484542
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Pred. y
 count    268.000000
mean       0.589552
std        0.492835
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: p1, dtype: float64
[[97 71]
 [13 87]]
              precision    recall  f1-score   support

           0       0.88      0.58      0.70       168
           1       0.55      0.87      0.67       100

    accuracy                           0.69       268
   macro avg       0.72      0.72      0.69       268
weighted avg       0.76      0.69      0.69       268



#**Umbral de 0.4**

- Exactitud (Accuracy): 0.79 (79%)

###Clase 0 (No sobrevive):
- Precisión: 0.86
- Tasa de recuperación (Recall): 0.79
- F1-score: 0.83
###Clase 1 (Sobrevive):
- Precisión: 0.69
- Tasa de recuperación (Recall): 0.79
- F1-score: 0.74

El umbral de 0.4 mejora la exactitud de 0.69 a 0.79. 10% de mejora comparado con el otro umbral.

In [ ]:
threshold = 0.4
proba = model.predict_proba(X_test)
#print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
#print(real_vs_pred)
print("Matriz de Confusión\n ",confusion_matrix(y_test, y_pred))
print("**Informe de Clasificación**\n ",classification_report(y_test, y_pred))

Real y
 count    268.000000
mean       0.373134
std        0.484542
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Pred. y
 count    268.000000
mean       0.425373
std        0.495324
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: p1, dtype: float64
Matriz de Confusión
  [[133  35]
 [ 21  79]]
**Informe de Clasificación**
                precision    recall  f1-score   support

           0       0.86      0.79      0.83       168
           1       0.69      0.79      0.74       100

    accuracy                           0.79       268
   macro avg       0.78      0.79      0.78       268
weighted avg       0.80      0.79      0.79       268



#**Umbral de 0.6**

- Exactitud (Accuracy): 0.81 (81%)

###Clase 0 (No sobrevive):
- Precisión: 0.81
- Tasa de recuperación (Recall): 0.90
- F1-score: 0.85
###Clase 1 (Sobrevive):
- Precisión: 0.79
- Tasa de recuperación (Recall): 0.65
- F1-score: 0.71

El umbral de 0.6 mejora la exactitud de 0.79 a 0.81. 2% de mejora comparado con el umbral de 0.4.

In [ ]:
threshold = 0.6
proba = model.predict_proba(X_test)
#print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
#print(real_vs_pred)
print("Matriz de Confusión\n ",confusion_matrix(y_test, y_pred))
print("**Informe de Clasificación**\n ",classification_report(y_test, y_pred))

Real y
 count    268.000000
mean       0.373134
std        0.484542
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Pred. y
 count    268.000000
mean       0.305970
std        0.461679
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: p1, dtype: float64
Matriz de Confusión
  [[151  17]
 [ 35  65]]
**Informe de Clasificación**
                precision    recall  f1-score   support

           0       0.81      0.90      0.85       168
           1       0.79      0.65      0.71       100

    accuracy                           0.81       268
   macro avg       0.80      0.77      0.78       268
weighted avg       0.80      0.81      0.80       268



#**Umbral de 0.8**

- Exactitud (Accuracy): 0.72 (72%)

###Clase 0 (No sobrevive):
- Precisión: 0.72
- Tasa de recuperación (Recall): 1.00
- F1-score: 0.84
###Clase 1 (Sobrevive):
- Precisión: 1.00
- Tasa de recuperación (Recall): 0.34
- F1-score: 0.51

El umbral de 0.8 es de lejos el más el incompleto comparado con los demás resultados de los umbrales.

In [ ]:
threshold = 0.8
proba = model.predict_proba(X_test)
#print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
#print(real_vs_pred)
print("Matriz de Confusión\n ",confusion_matrix(y_test, y_pred))
print("**Informe de Clasificación**\n ",classification_report(y_test, y_pred))

Real y
 count    268.000000
mean       0.373134
std        0.484542
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Pred. y
 count    268.000000
mean       0.111940
std        0.315883
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: p1, dtype: float64
Matriz de Confusión
  [[168   0]
 [ 70  30]]
**Informe de Clasificación**
                precision    recall  f1-score   support

           0       0.71      1.00      0.83       168
           1       1.00      0.30      0.46       100

    accuracy                           0.74       268
   macro avg       0.85      0.65      0.64       268
weighted avg       0.82      0.74      0.69       268



#### Definir conjunto de validación
70%, 15%, 15%
¿Cuántos ejemplos tiene el conjunto de validación y cuántos el conjunto de prueba?



In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=100)

# Dividir el conjunto de prueba/validación en prueba (15%) y validación (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=100)

# Imprimir los tamaños de los conjuntos de validación y prueba
print('Validación:', y_val.shape[0])
print('Prueba:', y_test.shape[0])

Validación: 134
Prueba: 134


#### Evaluar para diferentes umbrales
¿Con qué umbral se obtiene el mayor valor de exactitud?
###**Umbral 0.4:**

- Precisión para la clase 0 (no sobrevivió): 0.71
- Recall para la clase 0: 0.75
- F1-Score para la clase 0: 0.73
- Precisión para la clase 1 (sobrevivió): 0.65
- Recal para la clase 1: 0.61
- F1-Score para la clase 1: 0.63

  ### **Exactitud (accuracy): 0.69 (69%)**
###**Umbral 0.3:**

- Precisión para la clase 0: 0.75
- Recall para la clase 0: 0.71
- F1-Score para la clase 0: 0.73
- Precisión para la clase 1: 0.65
- Recal para la clase 1: 0.69
- F1-Score para la clase 1: 0.67

 ### **Exactitud: 0.70 (70%)**
###**Umbral 0.2:**

- Precisión para la clase 0: 0.76
- Recall para la clase 0: 0.56
- F1-Score para la clase 0: 0.65
- Precisión para la clase 1: 0.58
- Recal para la clase 1: 0.78
- F1-Score para la clase 1: 0.67
 ### **Exactitud: 0.66 (66%)**
###**Umbral 0.1:**
- Precisión para la clase 0: 1.00
- Recall para la clase 0: 0.15
- F1-Score para la clase 0: 0.26
- Precisión para la clase 1: 0.48
- Recal para la clase 1: 1.00
- F1-Score para la clase 1: 0.65
 ### **Exactitud: 0.52 (52%)**
  
- **Mejor Umbral:** El umbral de 0.3 parece ofrecer la mejor combinación de precisión y recall para ambas clases, resultando en la mayor exactitud general (0.70\70%). Pero el umbral de 0.4, tampoco es un umbral malo, tambien existe una buena combinación, además de que su exactitud es de 69%, los demás umbrales serían para casos especificos.

In [ ]:
proba = model.predict_proba(X_val)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
for threshold in [0.4, 0.3, 0.2, 0.1]:
  print('Umbral:', threshold)
  y_pred = (proba['p1'] >= threshold)
  y_pred.replace({True:1, False:0}, inplace=True)
  print(confusion_matrix(y_val, y_pred))
  print(classification_report(y_val, y_pred))

Umbral: 0.4
[[56 19]
 [23 36]]
              precision    recall  f1-score   support

           0       0.71      0.75      0.73        75
           1       0.65      0.61      0.63        59

    accuracy                           0.69       134
   macro avg       0.68      0.68      0.68       134
weighted avg       0.68      0.69      0.69       134

Umbral: 0.3
[[53 22]
 [18 41]]
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        75
           1       0.65      0.69      0.67        59

    accuracy                           0.70       134
   macro avg       0.70      0.70      0.70       134
weighted avg       0.70      0.70      0.70       134

Umbral: 0.2
[[42 33]
 [13 46]]
              precision    recall  f1-score   support

           0       0.76      0.56      0.65        75
           1       0.58      0.78      0.67        59

    accuracy                           0.66       134
   macro avg       0.67      0.67  

## Evaluación
Evalúe el modelo de mayor exactitud en el conjunto de validación sobre el conjunto de prueba.
Analice los resultados. ¿Cómo se comparan con los obtenidos para el conjunto de validación?

Compare los resultados obtenidos por la regresión logística con los de la línea base.

###**Modelo de Regresión Logística en el conjunto de prueba (umbral 0.4):**
  ### **Clase 0 (No Sobrevivió):**
- Precisión: 0.91
- Recall: 0.87
- F1-Score: 0.89
- Soporte: 84

### **Clase 1 (Sobrevivió):**

- Precisión: 0.80
- Recall: 0.86
- F1-Score: 0.83
- Soporte: 50

### **Exactitud General: 0.87 (87%)**

###**Modelo de Línea Base en el conjunto de prueba:**
### **Clase 0 (No Sobrevivió):**
- Precisión: 0.63
- Recall: 1.00
- F1-Score: 0.77
- Soporte: 168
### **Clase 1 (Sobrevivió):**
- Precisión: 0.00
- Recall: 0.00
- F1-Score: 0.00
- Soporte: 100

### **Exactitud General: 0.63 (63%)**

###**Precisión para la clase 0 (no sobrevivió):**
- Modelo de Regresión Logística: 0.83
- Línea Base: 0.63
###**Precisión para la clase 1 (sobrevivió):**
- Modelo de Regresión Logística: 0.64
- Línea Base: 0.00
###**Exactitud:**
- Modelo de Regresión Logística: 0.75
- Línea Base: 0.63

La regresión logística con umbral de 0.4 supera claramente a la línea base en todos los aspectos. Mientras que la línea base simplemente clasifica todos los ejemplos en la clase mayoritaria, la regresión logística identifica mejor a los sobrevivientes como a los no sobrevivientes con una alta precisión y recall. Es por ello que es mejor siempre elegir modelos trabajados que una simple linea base.

In [ ]:
proba = model.predict_proba(X_test)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
threshold = 0.4
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[73 11]
 [ 7 43]]
              precision    recall  f1-score   support

           0       0.91      0.87      0.89        84
           1       0.80      0.86      0.83        50

    accuracy                           0.87       134
   macro avg       0.85      0.86      0.86       134
weighted avg       0.87      0.87      0.87       134



#**Referencias**
- Ortega, C. (2023, February 23). ¿Qué es el coeficiente de correlación de Pearson? QuestionPro. https://www.questionpro.com/blog/es/coeficiente-de-correlacion-de-pearson/
- Pérez, R. G., Pino, G. G., Ballester, D. G., & Moreno, R. G. (2010). Modelo de regresión logística para estimar la dependencia según la escala de Lawton y Brody. SEMERGEN. Sociedad Española De Medicina Rural Y Generalista, 36(7), 365–371. https://doi.org/10.1016/j.semerg.2010.03.004
- SPSS Statistics Subscription - Classic. (n.d.). https://www.ibm.com/docs/es/spss-statistics/saas?topic=regression-logistic
- Mejores transformaciones de variables numéricas para ML. (2024, April 13). Datos 🥷. https://datos.ninja/tutorial/tecnicas-de-normalizacion-de-variables-numericas-con-python/
- GeeksforGeeks. (2024, March 21). One hot encoding in machine learning. GeeksforGeeks. https://www.geeksforgeeks.org/ml-one-hot-encoding/


#**Notas Adicionales**
En el desarrollo de este proyecto, utilizamos herramientas de inteligencia artificial para mejorar la redacción, resolver dudas sobre bibliotecas, sintaxis y otros aspectos generales. Estas herramientas nos facilitaron el proceso de aprendizaje, proporciona, fuentes donde indagar más como lo muestra las referencias,pero no proporcionaron soluciones específicas ni automatizaron tareas críticas. Todo el trabajo fue realizado en colaboración con el equipo, asegurando que cada integrante participara activamente en la construcción y desarrollo del ejercicio.